In [1]:
import os

In [2]:
%pwd

'e:\\My Projects\\NLP\\Projects\\LanguageTranslation\\LanguageTranslationGrmtoEng\\reseach'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\My Projects\\NLP\\Projects\\LanguageTranslation\\LanguageTranslationGrmtoEng'

In [31]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_url: Path
    local_data_file : Path
    unzip_dir: Path


In [19]:
from src.LanguageTranslation.constants import *
from src.LanguageTranslation.utils.common import read_yaml, create_directories


In [32]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.paragms = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config  =DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir= config.unzip_dir
        )
        return data_ingestion_config
        

In [11]:
! pip install gdown 


ERROR: Could not find a version that satisfies the requirement gdown (from versions: none)
ERROR: No matching distribution found for gdown


In [25]:
import os
import gdown
import zipfile
from LanguageTranslation.logging import logger
from LanguageTranslation.utils.common import get_size

In [35]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [37]:
try:
    config= ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2024-02-23 10:12:16,984: INFO:common:yaml file: config\config.yaml loaded successfully] 
[2024-02-23 10:12:16,984: INFO:common:yaml file: params.yaml loaded successfully] 
[2024-02-23 10:12:16,984: INFO:common:created directory at: artifacts] 
[2024-02-23 10:12:16,984: INFO:common:created directory at: articats/data_ingestion] 
[2024-02-23 10:12:16,984: INFO:879383870:Downloading data from https://drive.google.com/file/d/1G6hn9Y7CubAFMHQPOLcdmoXAUrJgPplU/view?usp=sharing into file artifacts/data_ingestion/data.zip] 


Downloading...
From: https://drive.google.com/uc?/export=download&id=1G6hn9Y7CubAFMHQPOLcdmoXAUrJgPplU
To: e:\My Projects\NLP\Projects\LanguageTranslation\LanguageTranslationGrmtoEng\artifacts\data_ingestion\data.zip
100%|██████████| 8.45M/8.45M [00:02<00:00, 3.40MB/s]


[2024-02-23 10:12:22,746: INFO:879383870:Downloaded data from https://drive.google.com/file/d/1G6hn9Y7CubAFMHQPOLcdmoXAUrJgPplU/view?usp=sharing into file artifacts/data_ingestion/data.zip] 
